In [46]:
using PythonCall
using SparseArrays
using Test
pf = pyimport("pfapack.pfaffian")
cpf = pyimport("pfapack.ctypes.pfaffian").pfaffian
np = pyimport("numpy")
include("pfaffian.jl")

pfaffian_schur

In [47]:
N = 100

100

In [66]:
A = rand(N, N);
A = A .- transpose(A);

T = [Vector{Float64}, Int64, Float64]

# for i in 1:(size(A, 1)-2)
i = 1
    A0 = A[i+1:end, i]

    for i in eachindex(T)
        B1 = pyconvert(T[i], pf.householder_real(np.array(A0))[i-1])
        B2 = householder_real(A0)[i]
        display(@test B1 ≈ B2)
    end
# end

Test Passed

Test Passed

Test Passed

In [67]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [Vector{ComplexF64}, Int64, ComplexF64]

# for i in 1:(size(A, 1)-2)
i = 1
    A0 = A[i+1:end, i]

    for i in eachindex(T)
        B1 = pyconvert(T[i], pf.householder_complex(np.array(A0))[i-1])
        B2 = householder_complex(A0)[i]
        display(@test B1 ≈ B2)
    end
# end

Test Passed

Test Passed

Test Passed

In [50]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [Matrix{ComplexF64}, Matrix{ComplexF64}]

for i in eachindex(T)
    B1 = pyconvert(T[i], pf.skew_tridiagonalize(np.array(A))[i-1])
    B2 = skew_tridiagonalize(A)[i]
    display(@test B1 ≈ B2)
end

Test Passed

Test Passed

In [51]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [Matrix{ComplexF64}, Matrix{ComplexF64}, SparseMatrixCSC{Float64, Int64}]

for i in eachindex(T)[1:end-1]
    B1 = pyconvert(T[i], pf.skew_LTL(np.array(A))[i-1])
    B2 = skew_LTL(A)[i]
    display(@test B1 ≈ B2)
end
@test Matrix(skew_LTL(A)[3]) ≈ pyconvert(Matrix{Float64}, pf.skew_LTL(np.array(A))[2].toarray())

Test Passed

Test Passed

Test Passed

In [52]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [ComplexF64]

B1 = pyconvert(T[1], pf.pfaffian_LTL(np.array(A)))
B2 = pfaffian_LTL(A)
display(@test B1 ≈ B2)

Test Passed

In [53]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [ComplexF64]

B1 = pyconvert(T[1], pf.pfaffian_householder(np.array(A)))
B2 = pfaffian_householder(A)
display(@test B1 ≈ B2)

Test Passed

In [54]:
A = rand(Float64, N, N);
A = A .- transpose(A);

T = [ComplexF64]

B1 = pyconvert(T[1], pf.pfaffian_schur(np.array(A)))
B2 = pfaffian_schur(A)
display(@test B1 ≈ B2)

Test Passed

In [57]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [ComplexF64]

B1 = pyconvert(T[1], pf.pfaffian(np.array(A), "P"))
B2 = pfaffian(A, method="P")
display(@test B1 ≈ B2)

B3 = det(A)
display(@test B1^2 ≈ B3)
display(@test B2^2 ≈ B3)

Test Passed

Test Passed

Test Passed

In [58]:
A = rand(ComplexF64, N, N);
A = A .- transpose(A);

T = [ComplexF64]

B1 = pyconvert(T[1], pf.pfaffian(np.array(A), "H"))
B2 = pfaffian(A, method="H")
display(@test B1 ≈ B2)

B3 = det(A)
display(@test B1^2 ≈ B3)
display(@test B2^2 ≈ B3)

Test Passed

Test Passed

Test Passed

In [71]:
using BenchmarkTools

A = rand(ComplexF64, N, N);
A = A .- transpose(A);

@btime $pf.pfaffian($np.array($A), $"H")

  1.410 ms (26 allocations: 680 bytes)


Python: (7.743159565381864e+26-7.281842063984266e+25j)

In [72]:
@btime pfaffian($A, method=$"H")

  2.155 ms (1455 allocations: 21.36 MiB)


7.74315956538198e26 - 7.281842063983816e25im

In [73]:
@btime $pf.pfaffian($np.array($A), $"P")

  1.412 ms (26 allocations: 680 bytes)


Python: (7.743159565381864e+26-7.281842063984266e+25j)

In [74]:
@btime pfaffian($A, method=$"P")

  935.708 μs (901 allocations: 10.90 MiB)


7.743159565381971e26 - 7.28184206398396e25im

In [ ]:
@btime cpf($np.array($A), $"H")